In [1]:
#import the necessary modules 
%matplotlib inline 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
#import scipy
import sklearn
import itertools as it
from itertools import cycle 
import os.path as op
import timeit 
import json
from matplotlib import animation
import matplotlib.font_manager as font_manager
from collections import namedtuple
#from functools import partial
#from pathlib import Path


In [2]:

# Set plotting style
plt.style.use('seaborn-white')

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
#import matplotlib.pyplot as plt

In [3]:
%matplotlib widget

In [4]:
import multiprocessing as m_proc
m_proc.cpu_count()

4

### Now use MD Analysis to calculate no. of frames a center PLGA residues and terminal PLGA residue is with 4 Angstroms of BSA (1.2 nm restrained system)

Import MDAnalysis

In [5]:
# Import MDAnalysis
import MDAnalysis as mda
import MDAnalysis.analysis.distances as maa_dist

### First table will be total fractional contacts and oligomer occupancy values for each Rg value 

#### Distance-based analysis 

Find residues that have at least one atom within a cutoff $d = 4.0$ Angstrom near water molecules in BSA/water simulation

Calculate the number of surface bsa residues from a 1 ns BSA/water simulation

In [6]:
#Units of Angstroms 
dmax = 4.0 

In [7]:
#exp_bsa_atoms = np.any(dij_tri <= dmax, axis=1)

# Boolean arrays must be of the same shape as the initial dimensions of the array being indexed.
# In the most straightforward case, the boolean array has the same shape
# Unlike in the case of integer index arrays, in the boolean case,
# the result is a 1-D array containing all the elements in the indexed array corresponding
#       to all the true elements in the boolean array.

#bres_wit_dmax = prot[exp_bsa_atoms].residues

In [8]:
## Define a function that does this, Courtesy of MDAnalysis tutorial 
def get_protresd_list(prot_atoms, g2_atoms, dmax, universe):
    """Find all protein residues for which atoms that are within dmax. Here, contact is defined as any group two atoms
    that is within 4 Angstroms of the protein atom group (group 1). If that condition is satisfied, that residue is added to an array. 
    This calculation is only done on one frame.
    
    Inputs consisted of: 
    
    - prot_atoms: Atom group from MD Analysis containing the protein atoms 
    - g2_atoms: Atom group from MD Analysis for the second group 
    - dmax: Distance cutoff in Angstroms, used to define contact between the two groups 
    - universe: MD Analysis variable containing information for the enitre trajectory
    
    Output consists of: 
    
    - mk: numpy array containing the residues that did contact group atoms"""
    
    ro = len(prot_atoms) # number of protein residues  
    
    cl = len(g2_atoms) # numbers of atoms in group two  
    
    dij_tri = np.zeros(shape=(ro,cl)) # Initialize a matrix the whose dimensions are (ro, cl)
    
    # Use MD Analysis to find distance between the protein atom groups and group two groups 
    dij_tri = maa_dist.distance_array(prot_atoms.positions
                                      , g2_atoms.positions, box=universe.trajectory.ts.dimensions)
    
    # Get the indices of the atoms that meet the 4 angstroms cutoff 
    exp_prot_atoms = np.any(dij_tri <= dmax, axis=1)
    
    # Get the corresponding residues that contact group two atoms 
    mk = np.array(prot_atoms[exp_prot_atoms].residues)
    
    return mk

In [9]:
# Define a function that outputs a dictionary of AA protein number and frame counts, where 
# the inputs will be number of frames, universe, prot and group 2 atom group, and dmax 
# Write function that takes a no of total frames, protein atom positions and group 2 atoms positions 
# and gives a dictionary of protein AA as keys and frame count as values 

def aa_frmcount(prot_atoms, g2_atoms, dmax, universe, start, end):
    """This function will output a dictionary of AA protein residue number and its corresponding frame count and occupancy. 
    
    Inputs consisted of: 
    
    - prot_atoms: Atom group from MD Analysis containing the protein atoms 
    - g2_atoms: Atom group from MD Analysis for the second group 
    - dmax: Distance cutoff in Angstroms, used to define contact between the two groups 
    - universe: MD Analysis variable containing information for the enitre trajectory
    - start: Start of the trajectory in picoseconds
    - end: End of the trajectory in picosenconds 
    
    Output is consisted of: 
    
    - aa_dict: A dictionary whose keys are residues that contacted group two atoms within the trajectory and
               values are an array containing the frame count and occupancy. Here, occupancy is defined as the 
               frame count of each residue divided by the total number of frames in trajectory block. Frame count 
               is the number of frames that a group two atom was less than dmax distance from any protein atom. 
    """
    
    aa_dict = {} # Initialize a dictionary to store the output. 
    
    laa = np.zeros(shape=len(prot_atoms.residues)) # Initialize a numpy array filled with zeros whose dim is no. of protein residues 
    
    br = np.array(prot_atoms.residues) # Store protein residues as given from MD Analysis 
    
    # This for loop goes through the trajectory block
    for ts in universe.trajectory[start:end]: 
        
        count = 0  # start counter at zero 
        
        # Use the get_protresd_list function to get residues that contact the group two atoms 
        bsres = get_protresd_list(prot_atoms, g2_atoms, dmax, universe) 
        
        # If the output array is empty, then no residues were within 4 Angstroms of group two atoms 
        if bsres.size == 0: 
            pass
        elif bsres.size != 0: # If array is not empty
    
            count += 1 # add to counter
            
            # Go through each residue 
            for i in bsres.flat:
        
                res_ind = np.where(br == i)[0]  # Get index for stored residue array that matches the residues from output in get_protresd_list
            
                laa[res_ind[0]] = laa[res_ind[0]] + count # update frame count to the specific residure 
                
    fin_res = np.where(laa != 0)[0] # Get indexes for residues with frame counts that are not zero
    
    # Calculate occupancy for each residue and store in a dictionary 
    for i in fin_res.flat:
        aa_dict[str(list(prot_atoms.residues[i:i+1])[0])] = [laa[i], laa[i]/(end - start)]
    
    return aa_dict 

In [10]:
 def grptwocnt_aa(prot_atoms, g2_atoms, dmax, universe):
    """ This function calculates the number of PLGA monomers within 4 A of a BSA AA residue and 
    return a numpy array containing the no. of PLGA monomers for each BSA AA residue at one single frame. Make sure you use a pdb file 
    for system description to get unique chain identifiers. This calculation is only done on one frame. Universe is used, in case 
    the periodic boundary is present. 
    
    GROMACS command: gmx trjconv -s topol.tpr -f confout.gro  -o new_conf.pdb -dump 0
 
    Inputs consist of: 
    
    - prot_atoms: Atom group from MD Analysis containing the protein atoms 
    - g2_atoms: Atom group from MD Analysis for the second group 
    - dmax: Distance cutoff in Angstroms, used to define contact between the two groups
    - universe: MD Analysis variable containing information for the enitre trajectory
    
    Outputs consist of: 
    
    - np.array(co_grpaa): For each amino acid group type, the total number of group two residues are given in a numpy array. 
    
    - la: numpy array containing the no. of group two (PLGA) monomers that meet the distance for each protein residue.
          Some array values will be empty because some residues will not contact the protein. 
    
    """
    
    gg_dict = {} # Initialize a dictionary to store one of the outputs
    
    # Use MD Analysis distance array function to get matrix of dim (# of group two atoms, # of protein atoms) 
    plga_mon = maa_dist.distance_array(g2_atoms.positions, prot_atoms.positions, box=universe.trajectory.ts.dimensions)
    
    # Return the indices of the elements in plga_mon that are less than dmax and are non-zero. Output is a tuple
    plga_mon_resd = np.nonzero(plga_mon < dmax)
    
    #pr_pol = np.nonzero(dij_tri < dmax)
    #print(pr_pol)
    
    # get polymer residue numbers for each atom in contact with the protein atom group
    a = np.array(list(g2_atoms[plga_mon_resd[0]].resids))
    
    # get polymer segid numbers for each atom in contact with the protein atom group
    b = np.array(list(g2_atoms[plga_mon_resd[0]].segids))
    
    new_gt = np.array([a,b])
    
    # indices where there is a change in values within the resids and segids polymer matrix 
    res_nsgt = np.where(new_gt[0][:-1] != new_gt[0][1:])[0]

    # indices where there is a change in values within the resids and segids protein matrix 
    res_prot_gt = np.where(prot_atoms[plga_mon_resd[1]].resids[:-1] != prot_atoms[plga_mon_resd[1]].resids[1:])[0]

    # Get the polymer residue indexes not included in the prot residues index list 
    mi_sgt = res_nsgt[np.where(np.isin(res_nsgt,res_prot_gt) == False )]
    
    # Add it to the protein residue index list and sort in ascending order 
    act_gt = np.sort(np.append(res_prot_gt,mi_sgt))

    # Initialize zeros array to store the no. of PLGA residues that contact each protein residue 
    la = np.zeros(shape=len(prot_atoms.residues))
    
    for i in act_gt.flat: 
    
        m_i = prot[plga_mon_resd[1]].resids[i]
        #print(m_i)

        la[m_i-1] = la[m_i-1] + 1
        
    # Find all residues that have more than zero PLGA monomers 
    fin_res = np.where(la != 0)[0]
    
    # Save number of PLGA residues per AA 
    for i in fin_res.flat:
        gg_dict[str(list(prot_atoms.residues[i:i+1])[0])] = la[i]

    # Grouping of residues in Smith et al  
    aromatic_res = ['PHE', 'TRP', 'TYR', 'HIS']
    hydrophobic_res = ['ALA', 'ILE', 'LEU', 'VAL', 'GLY', 'PRO','PHE', 'TRP','MET']
    polar_res = ['ASN', 'CYS', 'GLN', 'SER', 'THR','TYR']
    neg_res = ['ASP', 'GLU']
    pos_res = ['ARG', 'HIS', 'LYS']

    frac_res = [neg_res, pos_res, polar_res, hydrophobic_res, aromatic_res]
    
    # For each amino acid type in frac_res, this code chunk saves the PLGA residue count in a list and sums them together to 
    # to get a total number of PLGA residues within the trajectory for each AA group in frac_res
    co_grpaa = []

    for row in frac_res:
        fr_list = []
        for j in range(len(row)):
            for key, value in gg_dict.items():
                if row[j] in key:
                    fr_list.append(value)
        co_grpaa.append(sum(fr_list))
        
    return np.array(co_grpaa), la

In [11]:
def gtwo_trjcnt(prot_atoms, g2_atoms, dmax, universe, start, end):
    """This function calcuates the average number of PLGA mononers per BSA AA group and no of PLGA residues for each AA for each traj. block. 
    
    Inputs consist of: 
    
    - prot_atoms: Atom group from MD Analysis containing the protein atoms 
    - g2_atoms: Atom group from MD Analysis for the second group 
    - dmax: Distance cutoff in Angstroms, used to define contact between the two groups
    - universe: MD Analysis variable containing information for the enitre trajectory
    - start: Beginning of trajectory block in picoseconds 
    - end: End of trajectory block in picoseconds 
    
    Outputs consist of: 
    
    - aa_dict: dicitionary containing mean and pop. standard deviation of the no. of polymer residues for each different AA type 
    - l_final: the average no. of PLGA residues for each residue for each traj length, shape is (no. of prot_atoms residues)
    
    """
    
    # Different amino acid types 
    sf_lb = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
    
    # Initialize empty dictionary 
    aa_dict = {}
    
    # Initialize matrix whose dim is (prot_res, length of traj. block)
    laa = np.zeros(shape=(len(prot_atoms.residues),end-start))

    trj_aa = np.zeros(shape=(len(sf_lb), end-start))
    
    l_final = np.zeros(shape=(len(prot_atoms.residues)))
    
    universe.trajectory[start]
    
    count = 0
    
    for ts in universe.trajectory[start:end]: 
    
        fr_pres, hh_matx = grptwocnt_aa(prot_atoms, g2_atoms, dmax, universe)
        
        # No. of polymer residues that were in contact with the protein at each frame 
        laa[:,count] = hh_matx
 
        # For each amino acid type, save total number of PLGA residues at each frame for each AA group
        trj_aa[:, count] = fr_pres
        
        count += 1
        
    # find indicies for residues that had > 1 polymer residue 
    al_res = np.where(laa != 0)[0]

    # Calculating the average no. of PLGA residues for each residue for each traj length 
    for i in al_res.flat:
        l_final[i] = np.mean(laa[i,:])

    # get mean and pop. standard deviation of the no. of polymer residues for each different AA type 
    for i in range(len(sf_lb)):
        aa_dict[sf_lb[i]] = [np.mean(trj_aa[i,:]), np.std(trj_aa[i,:])]
    
    return aa_dict, l_final

In [12]:
def frac_cont(frm_count_dict):
    """ This function specifically takes as input the dictionary output from aa_frmcount function and 
    outputs a dictionary, nlkts, where each key is an amino acid type and whose corresponding value is as follows: 
    
    - nlkts: dictionary output     
    # co_grpaa[i] (Total no. of contacts for each Amino acid type ) 
    # tp_cnt[i] (total no. of amino acids for each type of amino acid type) 
    # norm_list[i] (total no. of contacts normalized by the protein surface fraction)
    # cont_l[i]] (Normalized fraction of contacts)
    
    """
    
    a_a = ["GLY","ALA","VAL","LEU","ILE","MET","PHE","TRP","PRO","SER","THR","CYS","TYR","ASN","GLN","ASP"
               ,"GLU","LYS","ARG","HIS"]
    
    # Grouping of residues in Smith et al  
    aromatic_res = ['PHE', 'TRP', 'TYR', 'HIS']
    hydrophobic_res = ['ALA', 'ILE', 'LEU', 'VAL', 'GLY', 'PRO','PHE', 'TRP','MET']
    polar_res = ['ASN', 'CYS', 'GLN', 'SER', 'THR','TYR']
    neg_res = ['ASP', 'GLU']
    pos_res = ['ARG', 'HIS', 'LYS']

    frac_res = [neg_res, pos_res, polar_res, hydrophobic_res, aromatic_res]
    sf_lbl = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
    
    # For each amino acid type in frac_res, this code chunk saves the frame count in a list and sums them together to 
    # to get a total frame count within the trajectory for each AA group in frac_res
    co_grpaa = []

    for row in frac_res:
        fr_list = []
        for j in range(len(row)):
            for key, value in frm_count_dict.items():
                if row[j] in key:
                    fr_list.append(value[0])
        co_grpaa.append(sum(fr_list))
        
    # This chunk of code gets an AA count from the above list, in order 
    # to get a total number of residues that contact BSA
    cpl_l = []

    for i in range(len(a_a)):
        count = 0
        for key, value in frm_count_dict.items():
            if a_a[i] in key:
                count += 1
        cpl_l.append(a_a[i]+" "+str(count))   

    # For each AA type in frac_res, this code chunk saves the count for each AA within 4 Angstroms of a PLGA trimer 
    # in a list based on the order in frac_res, then sums the counts to get a total number of AA for each AA type 
    tp_cnt = []   
    
    for row in frac_res:
        nw_l = []
        for i in range(len(row)):
            for j in range(len(cpl_l)):
                if row[i] in cpl_l[j]:
                    nw_l.append(int(cpl_l[j][4:6]))
        tp_cnt.append(sum(nw_l))           
    
    # Get the total count of AA that are within 4 A of PLGA oligomer
    bsum = len(frm_count_dict.keys())
    
    # The code chunk normalized the frame count of each AA group type by the protein surface fraction 
    # of each amino acid type contacted by a polymer surrogate.
    norm_list = []
    for i in range(len(co_grpaa)):
        lk = tp_cnt[i]/bsum
        if lk == 0: 
            norm_list.append(co_grpaa[i])
        elif lk != 0:
            norm_list.append(co_grpaa[i]/(tp_cnt[i]/bsum))
            
    # This conde chunk calculates the fractional contact based on the normalized frame count 
    cont_l = []
    nsum = sum(norm_list)
    for i in range(len(norm_list)):
        cont_l.append(norm_list[i]/nsum)
    
    #Save values in a dictionary 
    # Legend: co_grpaa[i] (Total no. of contacts) 
    # tp_cnt[i] (total no. of amino acids for each type of contact) 
    # norm_list[i] (total no. of contacts normalized by the protein surface fraction)
    # cont_l[i]] (Normalized fraction of contacts)
    nlkts = {}
    for i in range(len(sf_lbl)):
        nlkts[sf_lbl[i]] = [co_grpaa[i], tp_cnt[i], norm_list[i], cont_l[i]]
        
    return nlkts

In [13]:
# I want a list of total fraction of contacts where length is determined by no. of blocks and a dictionary 
# of contact groups as keys and list of fractional contacts as values(length of list will be no. of blocks)
def bavg_frac_cnt(no_of_blks, prot_atoms, g2_atoms, dmax, universe, no_surf, begin, final):
    """ This function takes as inputs: 
    
    Inputs consist of: 
    
    - no_of_blks: number of blocks
    - prot_atoms: Atom group from MD Analysis containing the protein atoms 
    - g2_atoms: Atom group from MD Analysis for the second group 
    - dmax: Distance cutoff in Angstroms, used to define contact between the two groups 
    - universe: MD Analysis variable containing information for the enitre trajectory
    - no_surf: total number of protein surface residues 
    - begin: Beginning of trajectory block in picoseconds
    - final: End of trajectory block in picoseconds 
    
    Outputs consist of: 
    
    - ot_dab (dictionary): total fraction of contacts within the specified block
    - plga_dict (dictionary): average no of PLGA monomers and the std deviation for each block for each AA group type
    - aares_npa: the avg no of PLGA residues, (5,583) shape
    
    """

    n_size = (final - begin)/no_of_blks
    
    frcb = []
    
    ot_dab = {}
    universe.trajectory[begin]
    
    sf_lbl = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic", "total_frac"]

    # Initialize matrix for AA type for each block 
    blk_nparr = np.zeros(shape=((len(sf_lbl)-1),no_of_blks))
    
    # 
    plga_nparr = np.zeros(shape=((len(sf_lbl)-1),no_of_blks), dtype=object)
    
    aares_npa = np.zeros(shape=(no_of_blks), dtype=object)
    
    for i in range(no_of_blks):
        
        tpl = []
 
        start = universe.trajectory.frame
        print(start)
    
        end = int(start + n_size)
        print(end)
        
        # A dictionary of AA protein residue number and its corresponding frame count and occupancy for each block 
        hn_bcks = aa_frmcount(prot_atoms, g2_atoms, dmax, universe, start, end)
        
        # a dictionary where each key is an amino acid type and whose corresponding value is as follows:    
        # co_grpaa[i] (Total no. of contacts for each Amino acid type ) 
        # tp_cnt[i] (total no. of amino acids for each type of amino acid type) 
        # norm_list[i] (total no. of contacts normalized by the protein surface fraction)
        # cont_l[i]] (Normalized fraction of contacts)
        ff_dict = frac_cont(hn_bcks)
        
        # the average number of PLGA mononers per protein AA group and no of PLGA residues for each AA for each trajectory block 
        nk_dict, res_mat = gtwo_trjcnt(prot_atoms, g2_atoms, dmax, universe, start, end)
        
        # total number of protein residues that contact polymer residues 
        lk = len(hn_bcks.keys())
        
        # total fraction of contacts is calculated here    
        frcb.append(lk/no_surf)

        # Normalized fractional contacts for each trajectory block 
        for key, value in ff_dict.items():
            tpl.append(value[3])
            
        count = 0
        for key, value in nk_dict.items():
            plga_nparr[count,i] = np.array(value)
            count += 1
            
        blk_nparr[:,i] = tpl  
        
        # (5,583) shape, for each block and for each AA residue, the avg no of PLGA residues is recorded
        aares_npa[i] = res_mat
        
        universe.trajectory[end]
    
    # Save fractional contacts for each AA group type, each element in the value array corresponds to a block 
    # calculated value
    for i in range(len(sf_lbl)-1):
        ot_dab[sf_lbl[i]] = blk_nparr[i,:]
        
    # Save average no of PLGA monomers and the std deviation for each block for each AA group type into dictionary
    plga_dict = {}
    for i in range(len(sf_lbl)-1):
        plga_dict[sf_lbl[i]] = plga_nparr[i,:]
    
    # total fraction of contacts within the specified blocks    
    ot_dab[sf_lbl[5]] = np.array(frcb)   
    
    # output legend 
    # ot_dab (dictionary): total fraction of contacts within the specified blocks
    # plga_dict (dictionary): average no of PLGA monomers and the std deviation for each block for each AA group type
    # aares_npa: the avg no of PLGA residues, (5,583) shape
    
    return ot_dab, plga_dict, aares_npa

In [14]:
def prot_poly_cntmovie(prot_atoms, g2_atoms, dmax, universe, start, end):
    """This function calculates the contact matrix between protein AA residues and group 2 residues at each timestep 
    and returns a multi-dim numpy array saves contact map information for the entire trajectory block. A contact is saved as 
    a boolean value (0 or 1)
    
    Inputs consist of: 
    
    
    
    Output consists of: 
    
    
    
    """
    
    universe.trajectory[start]
    
    cnt_un = 0 
    
    pp_mat = np.zeros(shape=(end-start), dtype=object)
    
    for ts in universe.trajectory[start:end]:
    
        # Get distance matrix at each frame
        ro = len(prot_atoms)
        cl = len(g2_atoms)
        dij_tri = np.zeros(shape=(ro,cl))
        dij_tri = maa_dist.distance_array(g2_atoms.positions, prot_atoms.positions, box=universe.trajectory.ts.dimensions)
    
        matfr = np.zeros(shape=(len(prot_atoms.residues),len(g2_atoms.residues)))
    
        pr_pol = np.nonzero(dij_tri < dmax)
    
        # Get residue IDs based on atoms that contact protein 
        a = np.array(list(g2_atoms[pr_pol[0]].resids))
        
        # Get segment IDs based on atoms that contact protein
        b = np.array(list(g2_atoms[pr_pol[0]].segids))
        
        # Concatenate the two arrays together 
        new_l = np.array([a,b])
    
        # indices where there is a change in values within the resids and segids polymer matrix 
        res_nos = np.where(new_l[0][:-1] != new_l[0][1:])[0]

        # indices where there is a change in values within the resids and segids protein matrix 
        res_prot = np.where(prot_atoms[pr_pol[1]].resids[:-1] != prot_atoms[pr_pol[1]].resids[1:])[0]

        # Get the polymer residue indexes not included in the prot residues index list 
        mi_s = res_nos[np.where(np.isin(res_nos,res_prot) == False )]
    
        # Add it to the protein residue index list and sort in ascending order 
        act = np.sort(np.append(res_prot,mi_s))

        for i in act.flat: 
    
            m_i = prot_atoms[pr_pol[1]].resids[i]
            #print(m_i)
    
            r_id1 = new_l[0][i]
            #print(r_id1)
    
            r_seg1 = new_l[1][i]
            #print(r_seg1)

            # Ensure the right chain contact is accounted for. 
            ar_res = np.array([20, 40])
    
            # Chain identfiers
            un_polys = np.unique(g2_atoms.segids)
    
            sg_i = np.where(un_polys == r_seg1)[0][0]
            #print(sg_i)
    
            if un_polys[0] == r_seg1:
                m_j = int(r_id1)
            elif un_polys[1] == r_seg1: 
                m_j = ar_res[0] + int(r_id1)
            elif un_polys[2] == r_seg1:
                m_j = ar_res[1] + int(r_id1)
        
            matfr[m_i-1,m_j-1] = 1
        
        pp_mat[cnt_un] = matfr
    
        cnt_un += 1
        
    return pp_mat

In [15]:
def AA_list_org(lorg_list):
    
    """List elements need have 'GLY  XX' as string format, where XX reps the number of GLY residues. Output is a
    sorted list of 'AA XX' according to the below order.  """
    
    hydrophobic_res = ['ALA', 'ILE', 'LEU', 'VAL', 'GLY', 'PRO','PHE', 'TRP','MET']
    polar_res = ['ASN', 'CYS', 'GLN', 'SER', 'THR','TYR']
    neg_res = ['ASP', 'GLU']
    pos_res = ['ARG', 'HIS', 'LYS']

    all_res = [pos_res, neg_res, polar_res, hydrophobic_res]
    #Change order of residues before making the bar graph
    # (1) Positively charged
    # (2) Negatively charged
    # (3) Polar residues 
    # (4) Hydrophobic residues 
    
    # This chunk of code sorts the counts of each AA that have 1001 or 1002 frame count based 
    # on the AA order in all_res
    arr_list = []

    for row in all_res:
        for i in range(len(lorg_list)):
            for j in range(len(row)):
                if row[j] == lorg_list[i][0:3]:
                    arr_list.append(lorg_list[i])
                    
    #This chunk of code splits the list arr_list to makes the AA: count of 1001 or 1002 frames data plottable 
    f_list = []
    fn_list = []
    for i in range(len(arr_list)):
        f_list.append(arr_list[i][0:3])
        fn_list.append(int(arr_list[i][5:]))
        
    return f_list, fn_list

In [16]:
def middle_of_band(band_start, band_stop, plot_min=0, plot_max=60):
    half_way = (band_stop - band_start) / 2
    mid_band = band_start + half_way
    plot_fraction = (mid_band - plot_min) / (plot_max - plot_min)

    return plot_fraction

# 1.2 nm PLGA restrained Rg 100 ns trajectory

Load the rg = 1.2 nm (3 PLGA N = 20 oligomer/BSA system) 

### Calc. total fraction of contacts

In [17]:
# Set up the MD Simulation, Make sure you do gmx trjconv -s topol.tpr -f confout.gro  -o new_conf.pdb -dump 0 -n bsaplga_nk.ndx to generate 
# a new pdb file that contains unique chain identifiers 
u_pn20 = mda.Universe("../1.2nm_bsa_prod/k4000_plumed/new_conf.pdb", "../1.2nm_bsa_prod/k4000_plumed/pp_1_2nmres.xtc")

In [18]:
u_pn20

<Universe with 10148 atoms>

Check that we are on the first frame

In [19]:
u_pn20.trajectory.frame

0

In [20]:
pn20_len = len(u_pn20.trajectory)
pn20_len

10001

In [ ]:
# Select the center PLGA residue atoms, heavy atoms only
pn20_cent = u_pn20.select_atoms("resname PLG and not type H")
pn20_cent

In [ ]:
# Select the terminal PLGA residues, heavy atoms only 
pn20_term = u_pn20.select_atoms("resname sPLG tPLG and not type H")
pn20_term

In [21]:
# Select one polymer chain, heavy atoms only 
#all_pn20 = u_pn20.select_atoms("(resname sPLG PLG tPLG and segid B) and not type H")

#Select all the PLGA residues, heavy atoms only
all_pn20 = u_pn20.select_atoms("resname sPLG PLG tPLG and not type H")
all_pn20

<AtomGroup with 543 atoms>

In [22]:
# Select BSA residues, heavy atoms only 
prot = u_pn20.select_atoms("protein and not type H")
prot

<AtomGroup with 4653 atoms>

Calculate AA frame counts for PLGA residues, 1.2 nm RG restraint, 100ns trajectory 

In [ ]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_len - 1
s_time = timeit.default_timer()
h2di = aa_frmcount(prot, all_pn20, dmax, u_pn20, start, end)
timeit.default_timer() - s_time

In [ ]:
# Frame count and occupancy for each residue
h2di

In [ ]:
len(h2di.keys())

In [ ]:
a_a = ["GLY","ALA","VAL","LEU","ILE","MET","PHE","TRP","PRO","SER","THR","CYS","TYR","ASN","GLN","ASP"
               ,"GLU","LYS","ARG","HIS"]

In [ ]:
# This code chunk gets the BSA residues and their corresponding number in a pandas dataframe 
red_bsa = []
bh = np.arange(0,584)
for i in range(583):
    b_str = str(list(prot.residues[i:i+1]))
    if str(bh[i+1]) in b_str: 
        red_bsa.append(str(b_str[10:13])+" "+str(bh[i+1]))

In [ ]:
pr_res = list(prot.residues)
ss_res = [str(row) for row in pr_res]
rkg = {key:h2di[key][0] for key, value in h2di.items()}
plg_1_2nmaa = pd.DataFrame(data=ss_res, columns=["BSA_des_res"])
plg_1_2nmaa['mda_plga_frm_1.2nm'] = plg_1_2nmaa['BSA_des_res'].map(rkg)
plg_1_2nmaa['BSA_des_res'] = red_bsa
plg_1_2nmaa['mda_plga_frm_1.2nm'] = plg_1_2nmaa['mda_plga_frm_1.2nm'].replace('nan', np.nan).fillna(0)
plg_1_2nmaa.tail()

In [ ]:
# Read in data from the oputput of wrapper.sh, where the frame count is given for each BSA residue that was within 
# 4 angstroms of PLGA trimer 
wat_data = pd.read_csv('occ_BSA1ns.txt', sep=" ", header=None, usecols=None ,index_col=None)
wat_data.columns = ["BSA_res_no","No. of frames (VMD)"]
wat_data = wat_data.drop("BSA_res_no", axis=1)

pr_res = list(prot.residues)
ss_res = [str(row) for row in pr_res]

wat_data['BSA_des_res'] = ss_res
wat_data = wat_data[['BSA_des_res',"No. of frames (VMD)"]]
#wat_data.head()

# load MDAnalysis values from MDA_BSA1ns.txt file(129003 atoms SOL group was used to calc. frame counts for txt.
# file)
h2ob_dict = json.load(open("MDA_BSA1ns.txt"))
wat_data['Mda_frames'] = wat_data['BSA_des_res'].map(h2ob_dict)


# From MD Analysis
#Get the count of bsa residues that have 1001 or 1002 frames ( I ran a 1 ns NPT simulation of 1 BSA in water )

#aa_count = pd.DataFrame(data=a_a)
c_list = []

for i in range(len(a_a)):
    count = 0
    for index, row in wat_data.iterrows():
        if a_a[i] in row["BSA_des_res"]:
            if row['Mda_frames'] == 1001: 
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames']))) 
            elif row['Mda_frames'] == 1000:
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames'])))
    c_list.append(str(str(a_a[i])+"  "+str(count)))
    
#c_list

# From VMD
#Get the count of bsa residues that have 1001 or 1002 frames ( I ran a 1 ns NPT simulation of 1 BSA in water )

#aa_count = pd.DataFrame(data=a_a)
vmd_list = []

for i in range(len(a_a)):
    count = 0
    for index, row in wat_data.iterrows():
        if a_a[i] in row["BSA_des_res"]:
            if row["No. of frames (VMD)"] == 1001: 
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames']))) 
            elif row["No. of frames (VMD)"] == 1002:
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames'])))
    vmd_list.append(str(str(a_a[i])+"  "+str(count)))

# Main difference is that Alanine 583 is counted for all 1001 frames. It seems VMD is unable to calc dist for that res
#vmd_list

#hydrophobic_res = ['ALA', 'ILE', 'LEU', 'VAL', 'GLY', 'PRO','PHE', 'TRP','MET']
#polar_res = ['ASN', 'CYS', 'GLN', 'SER', 'THR','TYR']
#neg_res = ['ASP', 'GLU']
#pos_res = ['ARG', 'HIS', 'LYS']
# aromatic_res = ['PHE', 'TRP', 'TYR', 'HIS']
#all_res = [pos_res, neg_res, polar_res, hydrophobic_res]

# Put the AA count in a pandas dataframe 
dg , ji = AA_list_org(c_list)
aa_count = pd.DataFrame(data=dg, index=None, columns=['Amino_acids'])
new_lf = pd.Series(data=ji, index=None)
vmg, vmdj = AA_list_org(vmd_list)
n2lf = pd.Series(data=vmdj, index=None)
aa_count['No_of_surf_res (MDAnalysis)'] = new_lf
aa_count['No_of_surf_res (VMD)'] = n2lf

In [ ]:
apl_list = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plg_1_2nmaa.iterrows():
    if r_pl['mda_plga_frm_1.2nm'] != 0:
        apl_list.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_l = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_list)):
        if r_a['Amino_acids'] in apl_list[i]:
            count += 1
    cpl_l.append(count)      
        
aa_count['plga_1.2nm_100ns'] = cpl_l
aa_count

In [ ]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['plga_1.2nm_100ns'].sum()

In [ ]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

In [ ]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['plga_1.2nm_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

Calculate mean occupancy and the standard deviation for 1.2 nm trajectory

Numpy mean and std function was used to calculate mean occupancy and std dev using occ values from aa_frmcount output

In [ ]:
# Mean occupancy and std deviation 
ll_mo = [value[1] for key, value in h2di.items()]
print("Mean Occpancy (1.2 nm Rg): "+str(np.mean(ll_mo)), "Occ. std. dev.: "+str(np.std(ll_mo)))

### Calc. fractional contacts for each AA group type 

In [ ]:
cd_12nm = frac_cont(h2di)
cd_12nm

In [ ]:
cd = frac_cont(h2di)
kklh = []
for key, value in cd.items():
    kklh.append(value[1])
# Must substract aromatic residues, since they are already counted
sum(kklh) - cd['Aromatic'][1]

In [ ]:
start = 0
end = pn20_len - 1
aa_12nm, l_f12nm = gtwo_trjcnt(prot, all_pn20, dmax, u_pn20, start, end)

In [ ]:
aa_12nm

In [ ]:
l_f12nm.shape

In [ ]:
no_surf = aa_count['No_of_surf_res (MDAnalysis)'].sum()
no_surf

In [ ]:
fcnt_rg1_2nm, prgrp_1_2nm, aa_matx = bavg_frac_cnt(5, prot, all_pn20, dmax, u_pn20, no_surf, 0, 10000)

In [ ]:
no_surf

In [ ]:
fcnt_rg1_2nm

In [ ]:
prgrp_1_2nm

In [ ]:
fc_12nm_mean = np.array([np.mean(fcnt_rg1_2nm['Negative']), np.mean(fcnt_rg1_2nm['Positive'])
                        ,np.mean(fcnt_rg1_2nm['Polar']),np.mean(fcnt_rg1_2nm['Hydrophobic'])
                        , np.mean(fcnt_rg1_2nm['Aromatic'])])
fc_12nm_mean

In [ ]:
fc_12nm_std = np.array([np.std(fcnt_rg1_2nm['Negative']), np.std(fcnt_rg1_2nm['Positive'])
                       ,np.std(fcnt_rg1_2nm['Polar']),np.std(fcnt_rg1_2nm['Hydrophobic'])
                       , np.std(fcnt_rg1_2nm['Aromatic'])])
fc_12nm_std

In [ ]:
x_pos = np.arange(5)
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, fc_12nm_mean, yerr=fc_12nm_std, ecolor='black',capsize=5, color='c')
plt.title(r'Fractional Contacts 1.2 nm Rg restrained', fontsize=15)
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'Fractional Contacts', fontsize=15)
#plt.ylim(-1.9,0)
#font = font_manager.FontProperties(family='Arial', style='normal', size='14')
#plt.legend([r'$N_{PLGA}$ = 8: $L_{p}$ = 17.8 $\AA$ ± 1.5 $\AA$'], loc=3, frameon=0, fontsize=14, prop=font)
#plt.tick_params(labelsize=14)
#plt.text(5,-0.15,r'R$^{2}$ = 0.99', fontsize=15, color='blue')

### Total fraction of contacts: averages and std dev calc from 5 20ns blocks

In [ ]:
np.mean(fcnt_rg1_2nm['total_frac'])

In [ ]:
np.std(fcnt_rg1_2nm['total_frac'])

### Avg no. PLGA residues per BSA AA residue group 

In [ ]:
prgrp_1_2nm

In [ ]:
mean_12nm = np.zeros(shape=5)
std_12nm = np.zeros(shape=5)
count = 0
for key, value in prgrp_1_2nm.items():
    mpl_12nm = []
    var_12nm = []
    for i in prgrp_1_2nm[str(key)].flat:
        mpl_12nm.append(i[0])
        var_12nm.append((i[1])**2)
    
    # calc frac cont averages
    mean_12nm[count] = np.mean(mpl_12nm)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_12nm[count] = np.std(mpl_12nm)
    # std_12nm[count] = np.sqrt(np.sum(var_12nm)/5)
    
    count += 1


In [ ]:
mean_12nm

In [ ]:
std_12nm 

In [ ]:
x_pos = np.arange(5)
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, mean_12nm, yerr=std_12nm, ecolor='black',capsize=5)
plt.title(r'No. of PLGA residues 1.2 nm Rg restrained', fontsize=15)
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

### Protein/polymer contact map movie

In [ ]:
#Select all the PLGA residues, heavy atoms only 
all_pn20 = u_pn20.select_atoms("resname sPLG PLG tPLG and not type H")
all_pn20

In [ ]:
g2_d, fr_p, h_mat = grptwocnt_aa(prot, all_pn20, dmax, u_pn20)

In [ ]:
# Select one polymer chain, heavy atoms only 
all_pn20_B = u_pn20.select_atoms("(resname sPLG PLG tPLG and segid B) and not type H")

all_pn20_C = u_pn20.select_atoms("(resname sPLG PLG tPLG and segid B) and not type H")

In [23]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trj_ppmap_12nm = prot_poly_cntmovie(prot, all_pn20, dmax, u_pn20, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [ ]:
trj_ppmap_12nm[9999]

In [ ]:
np.save('1.2nm_res.npy', trj_ppmap_12nm)    # .npy extension is added if not given

In [24]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 1.2 nm res", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.93, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_12nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('1.2nm_res.mp4',writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
def com_plga_oligs(g2_atoms, n_mon):
    hj_1 = []
    hj_2 = []
    hj_3 = []
    for i in range(n_mon):
        if i == 0:
            absdif = np.abs(np.diff(np.equal(list(g2_atoms.resids),i+1).view(np.int8)))
            rep_pga = np.concatenate(([0], np.where(absdif == 1)[0]))
            rep_pga += 1
            #print(rep_pga)
            rpn20 = rep_pga.reshape(3,2)
            hj_1.append(g2_atoms[rpn20[0][0]:rpn20[0][1]].center_of_mass())
            hj_2.append(g2_atoms[rpn20[1][0]:rpn20[1][1]].center_of_mass())
            hj_3.append(g2_atoms[rpn20[2][0]:rpn20[2][1]].center_of_mass())
        elif i != 0 and i != 19:
            absdif = np.abs(np.diff(np.equal(list(g2_atoms.resids),i+1).view(np.int8)))
            rep_pga = np.where(absdif == 1)[0]+1
            #print(rep_pga)
            rpn20 = rep_pga.reshape(3,2)
            hj_1.append(g2_atoms[rpn20[0][0]:rpn20[0][1]].center_of_mass())
            hj_2.append(g2_atoms[rpn20[1][0]:rpn20[1][1]].center_of_mass())
            hj_3.append(g2_atoms[rpn20[2][0]:rpn20[2][1]].center_of_mass())
        elif i == 19: 
            absdif = np.abs(np.diff(np.equal(list(g2_atoms.resids),i+1).view(np.int8)))
            rep_pga = np.where(absdif == 1)[0]+1
            rep_pga = np.concatenate((rep_pga,[len(g2_atoms)]))
            #print(rep_pga)
            rpn20 = rep_pga.reshape(3,2)
            hj_1.append(g2_atoms[rpn20[0][0]:rpn20[0][1]].center_of_mass())
            hj_2.append(g2_atoms[rpn20[1][0]:rpn20[1][1]].center_of_mass())
            hj_3.append(g2_atoms[rpn20[2][0]:rpn20[2][1]].center_of_mass())
    oligs_cb = np.concatenate((np.array(hj_1), np.array(hj_2), np.array(hj_3)))
    
    return oligs_cb

In [ ]:
cm_12nmoligs = com_plga_oligs(all_pn20, 20)
#cm_12nmoligs

In [ ]:
len(cm_12nmoligs)

In [ ]:
prot.select_atoms("resid 409").center_of_mass()

In [ ]:
cm_12nmoligs[18]

In [ ]:
nbb = maa_dist.distance_array(prot.select_atoms("resid 409").center_of_mass(), cm_12nmoligs[18]
                              ,box=u_pn20.trajectory.ts.dimensions)
#np.nonzero(nbb < dmax)
nbb

In [ ]:
# I wanted to examine contacts between heavy atoms in a residue named NDP and residues 1 to 268, 
#using the first frame to define native contacts and a distance cutoff of 3.0 Ang, COM wont be under 4 A
u_pn20.trajectory[0]
for ts in u_pn20.trajectory[0:10000]:
    pr_12nm = []
    for i in range(583): 
    #print(prot.select_atoms("resid "+str(i+1)).resids)
        pr_12nm.append(prot.select_atoms("resid "+str(i+1)).center_of_mass())
    pr12nm_com = np.array(pr_12nm)
    
    cm_12nmoligs = com_plga_oligs(all_pn20, 20)
    nbb = maa_dist.distance_array(prot.select_atoms("resid 409").center_of_mass(), cm_12nmoligs[19]
                              ,box=u_pn20.trajectory.ts.dimensions)
    print(np.nonzero(nbb < dmax))
    #pr12nm_com

In [ ]:
# matrix containing the avg number of PLGA residues for each block for each amino acid 
np.where(aa_matx[0, 0] != 0)

# 1.2 nm PLGA unrestrained Rg 100 ns trajectory

Load the rg = 1.2 nm (3 PLGA N = 20 oligomer/BSA system)

In [25]:
# load the unrestrained trajectory 
pn12nm_nores = mda.Universe("../1.2nm_bsa_prod/res_off/new_conf1.2nm.pdb"
                      , "../1.2nm_bsa_prod/res_off/pp_12nmresoff.xtc")

Check that we are on the first frame

In [26]:
pn12nm_nores.trajectory.frame

0

In [27]:
pn20nr_len = len(pn12nm_nores.trajectory)
pn20nr_len

10001

In [28]:
#Select all the PLGA residues, heavy atoms only 
pn20_allnr = pn12nm_nores.select_atoms("resname sPLG PLG tPLG and not type H")
pn20_allnr

<AtomGroup with 543 atoms>

In [29]:
# Select BSA residues, heavy atoms only 
prot_nores = pn12nm_nores.select_atoms("protein and not type H")
prot_nores

<AtomGroup with 4653 atoms>

### Calc. total fraction of contacts

In [ ]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20nr_len - 1
snr_time = timeit.default_timer()
h2di_nr = aa_frmcount(prot_nores, pn20_allnr, dmax, pn12nm_nores, start, end)
timeit.default_timer() - snr_time

In [ ]:
len(h2di_nr.keys())

In [ ]:
pr_resnr = list(prot_nores.residues)
ss_resnr = [str(row) for row in pr_resnr]
rkg_12nr = {key:h2di_nr[key][0] for key, value in h2di_nr.items()}
plg_1_2nr = pd.DataFrame(data=ss_resnr, columns=["BSA_des_res"])
plg_1_2nr['mda_1.2nm_nores'] = plg_1_2nr['BSA_des_res'].map(rkg_12nr)
plg_1_2nr['BSA_des_res'] = red_bsa
plg_1_2nr['mda_1.2nm_nores'] = plg_1_2nr['mda_1.2nm_nores'].replace('nan', np.nan).fillna(0)
plg_1_2nr.head()

In [ ]:
apl_12nm_nr = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plg_1_2nr.iterrows():
    if r_pl['mda_1.2nm_nores'] != 0:
        apl_12nm_nr.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_12nm_nr = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_12nm_nr)):
        if r_a['Amino_acids'] in apl_12nm_nr[i]:
            count += 1
    cpl_12nm_nr.append(count)      
        
aa_count['plga_1.2nm_100ns_NR'] = cpl_12nm_nr
#aa_count.drop('No_of_surf_res (VMD)', axis=1, inplace=True)
aa_count

In [ ]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['plga_1.2nm_100ns_NR'].sum()

In [ ]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

In [ ]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['plga_1.2nm_100ns_NR'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

In [ ]:
# Mean occupancy and std deviation 
ll_mo12_nr = [value[1] for key, value in h2di_nr.items()]
print("Mean Occpancy (1.2 nm unrestrained Rg): "+str(np.mean(ll_mo12_nr)), "Occ. std. dev.: "+str(np.std(ll_mo12_nr)))

In [ ]:
cd_12nm_nr = frac_cont(h2di_nr)
cd_12nm_nr

### Calc. fractional contacts for each AA group type 

In [ ]:
fcnt1_2nm_nr, prgrp12nm_nr, aamatx_12nm_nr = bavg_frac_cnt(5, prot_nores, pn20_allnr, dmax,
                                                        pn12nm_nores, no_surf, 0, 10000)

In [ ]:
fcnt1_2nm_nr

In [ ]:
fc_12nmnr_mean = np.array([np.mean(fcnt1_2nm_nr['Negative']), np.mean(fcnt1_2nm_nr['Positive'])
                        ,np.mean(fcnt1_2nm_nr['Polar']),np.mean(fcnt1_2nm_nr['Hydrophobic'])
                        , np.mean(fcnt1_2nm_nr['Aromatic'])])
fc_12nmnr_mean

In [ ]:
fc_12nmnr_std = np.array([np.std(fcnt1_2nm_nr['Negative']), np.std(fcnt1_2nm_nr['Positive'])
                       ,np.std(fcnt1_2nm_nr['Polar']),np.std(fcnt1_2nm_nr['Hydrophobic'])
                       , np.std(fcnt1_2nm_nr['Aromatic'])])
fc_12nmnr_std

In [ ]:
x_pos = np.arange(5)
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, fc_12nmnr_mean, yerr=fc_12nmnr_std, ecolor='black',capsize=5, color='c')
plt.title(r'Fractional Contacts 1.2 nm Rg unrestrained', fontsize=15)
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'Fractional Contacts', fontsize=15)

### Total fraction of contacts: averages and std dev calc from 5 20ns blocks

In [ ]:
np.mean(fcnt1_2nm_nr['total_frac'])

In [ ]:
np.std(fcnt1_2nm_nr['total_frac'])

### Avg no. PLGA residues per BSA AA residue group 

In [ ]:
prgrp12nm_nr

In [ ]:
mean_12nm_nr = np.zeros(shape=5)
std_12nm_nr = np.zeros(shape=5)
count = 0
for key, value in prgrp12nm_nr.items():
    mpl_12nm_nr = []
    var_12nm_nr = []
    for i in prgrp12nm_nr[str(key)].flat:
        mpl_12nm_nr.append(i[0])
        var_12nm_nr.append((i[1])**2)
    
    # calc frac cont averages
    mean_12nm_nr[count] = np.mean(mpl_12nm_nr)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_12nm_nr[count] = np.std(mpl_12nm_nr)
    
    count += 1


In [ ]:
mean_12nm_nr

In [ ]:
std_12nm_nr

In [ ]:
#std_12nm_nr[4] = 1
std_12nm_nr

In [ ]:
x_pos = np.arange(5)
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, mean_12nm_nr, yerr=std_12nm_nr, ecolor='black',capsize=5)
plt.title(r'No. of PLGA residues 1.2 nm Rg unrestrained', fontsize=15)
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

### Protein/polymer contact map movie

In [30]:
trj_pp12nm_nr = prot_poly_cntmovie(prot_nores, pn20_allnr, dmax, pn12nm_nores, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [ ]:
np.save('1.2nm_NoRes.npy', trj_pp12nm_nr)    # .npy extension is added if not given

Redo movie

In [31]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 1.2 nm Unres", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.94, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_pp12nm_nr[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1.03, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('1.2nm_NoRes.mp4',writer='ffmpeg',fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# 1.5 nm PLGA restrained Rg 100 ns trajectory

Load the rg = 1.5 nm (3 PLGA N = 20 oligomer/BSA system) 

In [ ]:
# Set up the MD Simulation
u15nm_pn20 = mda.Universe("../1.5nm_bsa_prod/k4000_1.5nm/res_1.5nmconf.pdb"
                      , "../1.5nm_bsa_prod/k4000_1.5nm/1_5nmppnopbc.xtc")

In [ ]:
u15nm_pn20

In [ ]:
pn20_len15 = len(u15nm_pn20.trajectory)
pn20_len15

In [ ]:
#Select all the PLGA residues, heavy atoms only 
all15_pn20 = u15nm_pn20.select_atoms("resname sPLG PLG tPLG and not type H")
all15_pn20

In [ ]:
# Select BSA residues, heavy atoms only 
prot_15nm = u15nm_pn20.select_atoms("protein and not type H")
prot_15nm

In [ ]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_len15 - 1
s_time = timeit.default_timer()
h2di_15nm = aa_frmcount(prot_15nm, all15_pn20, dmax, u15nm_pn20, start, end)
timeit.default_timer() - s_time
h2di_15nm

In [ ]:
len(h2di_15nm.keys())

In [ ]:
pr_res15nm = list(prot_15nm.residues)
ss_res15nm = [str(row) for row in pr_res15nm]
rkg_15nm = {key:h2di_15nm[key][0] for key, value in h2di_15nm.items()}
plg_1_5nmaa = pd.DataFrame(data=ss_res15nm, columns=["BSA_des_res"])
plg_1_5nmaa['mda_plga_frm_1.5nm'] = plg_1_5nmaa['BSA_des_res'].map(rkg_15nm)
plg_1_5nmaa['BSA_des_res'] = red_bsa
plg_1_5nmaa['mda_plga_frm_1.5nm'] = plg_1_5nmaa['mda_plga_frm_1.5nm'].replace('nan', np.nan).fillna(0)
plg_1_5nmaa.head()

In [ ]:
apl_15nm = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plg_1_5nmaa.iterrows():
    if r_pl['mda_plga_frm_1.5nm'] != 0:
        apl_15nm.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_15nm = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_15nm)):
        if r_a['Amino_acids'] in apl_15nm[i]:
            count += 1
    cpl_15nm.append(count)      
        
aa_count['plga_1.5nm_100ns'] = cpl_15nm
#aa_count.drop('No_of_surf_res (VMD)', axis=1, inplace=True)
aa_count

In [ ]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['plga_1.5nm_100ns'].sum()

In [ ]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

In [ ]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['plga_1.5nm_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

In [ ]:
# Mean occupancy and std deviation 
ll_mo15 = [value[1] for key, value in h2di_15nm.items()]
print("Mean Occpancy (1.5 nm Rg): "+str(np.mean(ll_mo15)), "Occ. std. dev.: "+str(np.std(ll_mo15)))

In [ ]:
cd_15nm = frac_cont(h2di_15nm)
cd_15nm

### Calc. fractional contacts for each AA group type 

In [ ]:
fcnt_rg1_5nm, prgrp_1_5nm, aa_matx_15nm = bavg_frac_cnt(5, prot_15nm, all15_pn20, dmax,
                                                        u15nm_pn20, no_surf, 0, 10000)

In [ ]:
fcnt_rg1_5nm

In [ ]:
fc_15nm_mean = np.array([np.mean(fcnt_rg1_5nm['Negative']), np.mean(fcnt_rg1_5nm['Positive'])
                        ,np.mean(fcnt_rg1_5nm['Polar']),np.mean(fcnt_rg1_5nm['Hydrophobic'])
                        , np.mean(fcnt_rg1_5nm['Aromatic'])])
fc_15nm_mean

In [ ]:
fc_15nm_std = np.array([np.std(fcnt_rg1_5nm['Negative']), np.std(fcnt_rg1_5nm['Positive'])
                       ,np.std(fcnt_rg1_5nm['Polar']),np.std(fcnt_rg1_5nm['Hydrophobic'])
                       , np.std(fcnt_rg1_5nm['Aromatic'])])
fc_15nm_std

In [ ]:
x_pos = np.arange(5)
width = 0.35
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, fc_12nm_mean, width, yerr=fc_12nm_std, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, fc_15nm_mean, width, yerr=fc_15nm_std, ecolor='black',capsize=5, color='c')
plt.title(r'Fractional Contacts Rg restrained', fontsize=15)
plt.xticks(x_pos+width/2, labels=aa_types, fontsize=12)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm'], frameon=False)
plt.ylabel(r'Fractional Contacts', fontsize=15)

### Total fraction of contacts: averages and std dev calc from 5 20 ns blocks

In [ ]:
np.mean(fcnt_rg1_5nm['total_frac'])

In [ ]:
np.std(fcnt_rg1_5nm['total_frac'])

In [ ]:
prgrp_1_5nm

In [ ]:
# matrix containing the avg number of PLGA residues for each block for each amino acid 
np.where(aa_matx_15nm[0, 0] != 0)

### Avg no. PLGA residues per BSA AA residue group 

In [ ]:
prgrp_1_5nm

In [ ]:
mean_15nm = np.zeros(shape=5)
std_15nm = np.zeros(shape=5)
count = 0
for key, value in prgrp_1_5nm.items():
    mpl_15nm = []
    var_15nm = []
    for i in prgrp_1_5nm[str(key)].flat:
        mpl_15nm.append(i[0])
        var_15nm.append((i[1])**2)
    
    # calc frac cont averages
    mean_15nm[count] = np.mean(mpl_15nm)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_15nm[count] = np.std(mpl_15nm)
    #std_15nm[count] = np.sqrt(np.sum(var_15nm)/5)
    
    count += 1


In [ ]:
mean_15nm

In [ ]:
std_15nm

In [ ]:
x_pos = np.arange(5)
width = 0.35
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, mean_12nm, width, yerr=std_12nm, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, mean_15nm, width, yerr=std_15nm, ecolor='black',capsize=5, color='c')
plt.title(r'No. of PLGA residues Rg restrained', fontsize=15)
plt.xticks(x_pos+width/2, labels=aa_types, fontsize=12)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm'], frameon=False)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

### Protein/polymer contact map movie

In [ ]:
#Select all the PLGA residues, heavy atoms only 
all15_pn20 = u15nm_pn20.select_atoms("resname sPLG PLG tPLG and not type H")
all15_pn20

In [ ]:
# Select one polymer chain, heavy atoms only 
all_pn20_B = u_pn20.select_atoms("(resname sPLG PLG tPLG and segid B) and not type H")

all_pn20_C = u_pn20.select_atoms("(resname sPLG PLG tPLG and segid B) and not type H")

In [ ]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trj_ppmap_15nm = prot_poly_cntmovie(prot_15nm, all15_pn20, dmax, u15nm_pn20, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [ ]:
np.save('1.5nm_res.npy', trj_ppmap_15nm)    # .npy extension is added if not given

In [ ]:
for i in range(1000):
    print(np.nonzero(trj_ppmap_15nm[i]))

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 1.5 nm res.", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.94, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_15nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1.03, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('1.5nm_res.mp4', writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()

# 1.5 nm PLGA unrestrained Rg 100 ns trajectory

Load the rg = 1.2 nm (3 PLGA N = 20 oligomer/BSA system)

In [ ]:
# load the unrestrained trajectory 
pn15nm_nores = mda.Universe("../1.5nm_bsa_prod/1.5res_off/new_conf1.5nm.pdb"
                      , "../1.5nm_bsa_prod/1.5res_off/pp1_5nmresoff.xtc")

Check that we are on the first frame

In [ ]:
pn15nm_nores.trajectory.frame

In [ ]:
pn20_15nr = len(pn15nm_nores.trajectory)
pn20_15nr

In [ ]:
#Select all the PLGA residues, heavy atoms only 
pn20_all15nr = pn15nm_nores.select_atoms("resname sPLG PLG tPLG and not type H")
pn20_all15nr

In [ ]:
# Select BSA residues, heavy atoms only 
prot15_nores = pn15nm_nores.select_atoms("protein and not type H")
prot15_nores

### Calc. total fraction of contacts

In [ ]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_15nr - 1
s_time15nr = timeit.default_timer()
h2di_15nr = aa_frmcount(prot15_nores, pn20_all15nr, dmax, pn15nm_nores, start, end)
timeit.default_timer() - s_time15nr

In [ ]:
len(h2di_15nr.keys())

In [ ]:
pr_15nr = list(prot15_nores.residues)
ss_15nr = [str(row) for row in pr_15nr]
rkg_15nm_nr = {key:h2di_15nr[key][0] for key, value in h2di_15nr.items()}
plg_1_5nm_nr = pd.DataFrame(data=ss_15nr, columns=["BSA_des_res"])
plg_1_5nm_nr['mda_1.5nm_nr'] = plg_1_5nm_nr['BSA_des_res'].map(rkg_15nm_nr)
plg_1_5nm_nr['BSA_des_res'] = red_bsa
plg_1_5nm_nr['mda_1.5nm_nr'] = plg_1_5nm_nr['mda_1.5nm_nr'].replace('nan', np.nan).fillna(0)
plg_1_5nm_nr.head()

In [ ]:
apl_15nm_nr = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plg_1_5nm_nr.iterrows():
    if r_pl['mda_1.5nm_nr'] != 0:
        apl_15nm_nr.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_15nm_nr = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_15nm_nr)):
        if r_a['Amino_acids'] in apl_15nm_nr[i]:
            count += 1
    cpl_15nm_nr.append(count)      
        
aa_count['plga_1.5nm_100ns_NR'] = cpl_15nm_nr
#aa_count.drop('No_of_surf_res (VMD)', axis=1, inplace=True)
aa_count

In [ ]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['plga_1.5nm_100ns_NR'].sum()

In [ ]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

In [ ]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['plga_1.5nm_100ns_NR'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

In [ ]:
# Mean occupancy and std deviation 
ll_mo15_nr = [value[1] for key, value in h2di_15nr.items()]
print("Mean Occpancy (1.5 nm Rg Unrestrained): "+str(np.mean(ll_mo15_nr)), "Occ. std. dev.: "+str(np.std(ll_mo15_nr)))

In [ ]:
cd_15nr = frac_cont(h2di_15nr)
cd_15nr

### Calc. fractional contacts for each AA group type 

In [ ]:
fcntrg1_5nm_nr, prgrp1_5nm_nr, aamatx_15nm_nr = bavg_frac_cnt(5, prot15_nores, pn20_all15nr, dmax,
                                                        pn15nm_nores, no_surf, 0, 10000)

In [ ]:
fcntrg1_5nm_nr

In [ ]:
fc15nm_mean_nr = np.array([np.mean(fcntrg1_5nm_nr['Negative']), np.mean(fcntrg1_5nm_nr['Positive'])
                        ,np.mean(fcntrg1_5nm_nr['Polar']),np.mean(fcntrg1_5nm_nr['Hydrophobic'])
                        , np.mean(fcntrg1_5nm_nr['Aromatic'])])
fc15nm_mean_nr

In [ ]:
fc15nm_std_nr = np.array([np.std(fcntrg1_5nm_nr['Negative']), np.std(fcntrg1_5nm_nr['Positive'])
                       ,np.std(fcntrg1_5nm_nr['Polar']),np.std(fcntrg1_5nm_nr['Hydrophobic'])
                       , np.std(fcntrg1_5nm_nr['Aromatic'])])
fc15nm_std_nr

In [ ]:
x_pos = np.arange(5)
width = 0.35
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, fc_12nmnr_mean, width, yerr=fc_12nmnr_std, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, fc15nm_mean_nr, width, yerr=fc15nm_std_nr, ecolor='black',capsize=5, color='c')
plt.title(r'Fractional Contacts Rg unrestrained', fontsize=15)
plt.xticks(x_pos+width/2, labels=aa_types, fontsize=12)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm'], frameon=False)
plt.ylabel(r'Fractional Contacts', fontsize=15)

### Total fraction of contacts: averages and std dev calc from 5 20ns blocks

In [ ]:
np.mean(fcntrg1_5nm_nr['total_frac'])

In [ ]:
np.std(fcntrg1_5nm_nr['total_frac'])

### Avg no. PLGA residues per BSA AA residue group 

In [ ]:
prgrp1_5nm_nr

In [ ]:
mean_15nm_nr = np.zeros(shape=5)
std_15nm_nr = np.zeros(shape=5)
count = 0
for key, value in prgrp1_5nm_nr.items():
    mpl_15nm_nr = []
    var_15nm_nr = []
    for i in prgrp1_5nm_nr[str(key)].flat:
        mpl_15nm_nr.append(i[0])
        var_15nm_nr.append((i[1])**2)
    
    # calc frac cont averages
    mean_15nm_nr[count] = np.mean(mpl_15nm_nr)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_15nm_nr[count] = np.std(mpl_15nm_nr)
    #std_15nm_nr[count] = np.sqrt(np.sum(var_15nm_nr)/5)
    
    count += 1


In [ ]:
mean_15nm_nr

In [ ]:
std_15nm_nr

In [ ]:
x_pos = np.arange(5)
width = 0.35
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, mean_12nm_nr, width, yerr=std_12nm_nr, ecolor='black',capsize=5, color='green')
plt.bar(x_pos+width, mean_15nm_nr, width, yerr=std_15nm_nr, ecolor='black',capsize=5, color='violet')
plt.title(r'No. of PLGA residues Rg unrestrained', fontsize=15)
plt.xticks(x_pos+width/2, labels=aa_types, fontsize=12)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm'], frameon=False)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

### Protein/polymer contact map movie

In [ ]:
trj_ppmap15nm_nr = prot_poly_cntmovie(prot15_nores, pn20_all15nr, dmax, pn15nm_nores, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [ ]:
np.save('1.5nm_NoRes.npy', trj_ppmap15nm_nr)    # .npy extension is added if not given

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 1.5 nm Unres", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.94, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap15nm_nr[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1.03, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('1.5nm_NoRes.mp4', writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()

# 2 nm restrained Rg PLGA 100 ns trajectory

Load the rg = 1.5 nm (3 PLGA N = 20 oligomer/BSA system) 

### Calc total fraction of contacts 

In [ ]:
# Set up the MD Simulation
u2nm_pn20 = mda.Universe("../2nm_bsa_prod/k4000_2nm/new_conf2nm.pdb"
                      , "../2nm_bsa_prod/k4000_2nm/nopbc_ppsys.xtc")

In [ ]:
u2nm_pn20

In [ ]:
pn20_len2nm = len(u2nm_pn20.trajectory)
pn20_len2nm

In [ ]:
#Select all the PLGA residues, heavy atoms only 
all2nm_pn20 = u2nm_pn20.select_atoms("resname sPLG PLG tPLG and not type H")
all2nm_pn20

In [ ]:
# Select BSA residues, heavy atoms only 
prot_2nm = u2nm_pn20.select_atoms("protein and not type H")
prot_2nm

In [ ]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_len2nm - 1
s_time = timeit.default_timer()
h2di_2nm = aa_frmcount(prot_2nm, all2nm_pn20, dmax, u2nm_pn20, start, end)
timeit.default_timer() - s_time
h2di_2nm

In [ ]:
len(h2di_2nm.keys())

In [ ]:
pr_res2nm = list(prot_2nm.residues)
ss_res2nm = [str(row) for row in pr_res2nm]
rkg_2nm = {key:h2di_2nm[key][0] for key, value in h2di_2nm.items()}
plg_2nmaa = pd.DataFrame(data=ss_res2nm, columns=["BSA_des_res"])
plg_2nmaa['mda_plga_frm_2nm'] = plg_2nmaa['BSA_des_res'].map(rkg_2nm)
plg_2nmaa['BSA_des_res'] = red_bsa
plg_2nmaa['mda_plga_frm_2nm'] = plg_2nmaa['mda_plga_frm_2nm'].replace('nan', np.nan).fillna(0)
plg_2nmaa.head()

In [ ]:
apl_2nm = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plg_2nmaa.iterrows():
    if r_pl['mda_plga_frm_2nm'] != 0:
        apl_2nm.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_2nm = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_2nm)):
        if r_a['Amino_acids'] in apl_2nm[i]:
            count += 1
    cpl_2nm.append(count)      
        
aa_count['plga_2nm_100ns'] = cpl_2nm
#aa_count.drop('No_of_surf_res (VMD)', axis=1, inplace=True)
aa_count

In [ ]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['plga_2nm_100ns'].sum()

In [ ]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

In [ ]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['plga_2nm_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

In [ ]:
# Mean occupancy and std deviation 
ll_mo2nm = [value[1] for key, value in h2di_2nm.items()]
print("Mean Occpancy (2 nm Rg): "+str(np.mean(ll_mo2nm)), "Occ. std. dev.: "+str(np.std(ll_mo2nm)))

In [ ]:
cd_2nm = frac_cont(h2di_2nm)
cd_2nm

### Calc. fractional contacts for each AA group type 

In [ ]:
fcnt_rg2nm, prgrp_2nm, aa_matx_2nm = bavg_frac_cnt(5, prot_2nm, all2nm_pn20, dmax, u2nm_pn20, no_surf, 0, 10000)

In [ ]:
fcnt_rg2nm

In [ ]:
fc_2nm_mean = np.array([np.mean(fcnt_rg2nm['Negative']), np.mean(fcnt_rg2nm['Positive'])
                        ,np.mean(fcnt_rg2nm['Polar']),np.mean(fcnt_rg2nm['Hydrophobic'])
                        , np.mean(fcnt_rg2nm['Aromatic'])])
fc_2nm_mean

In [ ]:
fc_2nm_std = np.array([np.std(fcnt_rg2nm['Negative']), np.std(fcnt_rg2nm['Positive'])
                       ,np.std(fcnt_rg2nm['Polar']),np.std(fcnt_rg2nm['Hydrophobic'])
                       , np.std(fcnt_rg2nm['Aromatic'])])
fc_2nm_std

In [ ]:
x_pos = np.arange(5)
width = 0.28
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, fc_12nm_mean, width, yerr=fc_12nm_std, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, fc_15nm_mean, width, yerr=fc_15nm_std, ecolor='black',capsize=5, color='c')
plt.bar(x_pos+(2*width), fc_2nm_mean, width, yerr=fc_2nm_std, ecolor='black',capsize=5, color='lightslategray')
plt.title(r'Fractional Contacts Rg restrained', fontsize=15)
plt.xticks(x_pos+width, labels=aa_types, fontsize=12)
plt.ylim(0,0.4)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm', 'Rg = 2 nm'], frameon=False)
plt.ylabel(r'Fractional Contacts', fontsize=15)

### Total fraction of contacts: averages and std dev calc from 5 20 ns blocks

In [ ]:
np.mean(fcnt_rg2nm['total_frac'])

In [ ]:
np.std(fcnt_rg2nm['total_frac'])

In [ ]:
prgrp_2nm

In [ ]:
# matrix containing the avg number of PLGA residues for each block for each amino acid 
np.where(aa_matx_2nm[0, 0] != 0)

### Avg no. PLGA residues per BSA AA residue group 

In [ ]:
prgrp_2nm

In [ ]:
mean_2nm = np.zeros(shape=5)
std_2nm = np.zeros(shape=5)
count = 0
for key, value in prgrp_2nm.items():
    mpl_2nm = []
    var_2nm = []
    for i in prgrp_2nm[str(key)].flat:
        mpl_2nm.append(i[0])
        var_2nm.append((i[1])**2)
    
    # calc frac cont averages
    mean_2nm[count] = np.mean(mpl_2nm)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_2nm[count] = np.std(mpl_2nm)
    #std_2nm[count] = np.sqrt(np.sum(var_2nm)/5)
    
    count += 1


In [ ]:
mean_2nm

In [ ]:
std_2nm

In [ ]:
mean_15nm

In [ ]:
std_15nm

In [ ]:
x_pos = np.arange(5)
width = 0.28
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, mean_12nm, width, yerr=std_12nm, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, mean_15nm, width, yerr=std_15nm, ecolor='black',capsize=5, color='c')
plt.bar(x_pos+(2*width), mean_2nm, width, yerr=std_2nm, ecolor='black',capsize=5, color='lightslategray')
plt.title(r'No. of PLGA residues Rg restrained', fontsize=15)
plt.xticks(x_pos+width, labels=aa_types, fontsize=12)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm', 'Rg = 2 nm'], frameon=False)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

In [ ]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trj_ppmap_2nm = prot_poly_cntmovie(prot_2nm, all2nm_pn20, dmax, u2nm_pn20, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [ ]:
np.save('2nm_res.npy', trj_ppmap_2nm)    # .npy extension is added if not given

In [ ]:
trj_load2nm_res = np.load('2nm_res.npy', allow_pickle=True)
trj_load2nm_res

In [ ]:
np.where(trj_load2nm_res[9920] == 0 )

In [ ]:
trj_load2nm_res[9920]

In [ ]:
trj_load2nm_res.shape

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 2 nm restrained", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.94, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_2nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1.03, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('2nm_res.mp4', writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()

Write a function that calculates the average total fraction of contacts(No of residues contacted by PLGA oilgs/total no of surface residues) and their standard deviations for a given number of trajectory blocks 

# 2 nm PLGA unrestrained Rg 100 ns trajectory

Load the rg = 1.2 nm (3 PLGA N = 20 oligomer/BSA system)

In [ ]:
# load the unrestrained trajectory 
pn2nm_nores = mda.Universe("../2nm_bsa_prod/2nmres_off/new_conf2nmnm.pdb"
                      , "../2nm_bsa_prod/2nmres_off/pp_resoffnopbc.xtc")

Check that we are on the first frame

In [ ]:
pn2nm_nores.trajectory.frame

In [ ]:
p2nm_len_nr = len(pn2nm_nores.trajectory)
p2nm_len_nr

In [ ]:
#Select all the PLGA residues, heavy atoms only 
p2nm_allnr = pn2nm_nores.select_atoms("resname sPLG PLG tPLG and not type H")
p2nm_allnr

In [ ]:
# Select BSA residues, heavy atoms only 
prot2nm_nores = pn2nm_nores.select_atoms("protein and not type H")
prot2nm_nores

### Calc. total fraction of contacts

In [ ]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = p2nm_len_nr - 1
s_time = timeit.default_timer()
h2di_2nr = aa_frmcount(prot2nm_nores, p2nm_allnr, dmax, pn2nm_nores, start, end)
timeit.default_timer() - s_time

In [ ]:
len(h2di_2nr.keys())

In [ ]:
pr_2nr = list(prot2nm_nores.residues)
ss_2nr = [str(row) for row in pr_2nr]
rkg_2nm = {key:h2di_2nr[key][0] for key, value in h2di_2nr.items()}
plg_2nm_nr = pd.DataFrame(data=ss_2nr, columns=["BSA_des_res"])
plg_2nm_nr['mda_2nm_nr'] = plg_2nm_nr['BSA_des_res'].map(rkg_2nm)
plg_2nm_nr['BSA_des_res'] = red_bsa
plg_2nm_nr['mda_2nm_nr'] = plg_2nm_nr['mda_2nm_nr'].replace('nan', np.nan).fillna(0)
plg_2nm_nr.head()

In [ ]:
apl_2nm_nr = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plg_2nm_nr.iterrows():
    if r_pl['mda_2nm_nr'] != 0:
        apl_2nm_nr.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_2nm_nr = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_2nm_nr)):
        if r_a['Amino_acids'] in apl_2nm_nr[i]:
            count += 1
    cpl_2nm_nr.append(count)      
        
aa_count['plga_2nm_100ns_NR'] = cpl_2nm_nr
#aa_count.drop('No_of_surf_res (VMD)', axis=1, inplace=True)
aa_count

In [ ]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['plga_2nm_100ns_NR'].sum()

In [ ]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

In [ ]:
# This gives the total fraction of contacts within the 2 nm unrestrained Rg 100 ns trajectory
aa_count['plga_2nm_100ns_NR'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

In [ ]:
# Mean occupancy and std deviation 
ll_mo2_nr = [value[1] for key, value in h2di_2nr.items()]
print("Mean Occpancy (2 nm Rg): "+str(np.mean(ll_mo2_nr)), "Occ. std. dev.: "+str(np.std(ll_mo2_nr)))

In [ ]:
cd_2nm = frac_cont(h2di_2nr)
cd_2nm

### Calc. fractional contacts for each AA group type 

In [ ]:
test_nr = aa_frmcount(prot2nm_nores, p2nm_allnr, dmax, pn2nm_nores, 6000, 8000)

In [ ]:
frac_cont(test_nr)

In [ ]:
no_surf

In [ ]:
fcntrg2nm_nr, prgrp2nm_nr, aamatx_2nm_nr = bavg_frac_cnt(5, prot2nm_nores, p2nm_allnr, dmax, pn2nm_nores, no_surf, 0, 10000)

In [ ]:
fcntrg2nm_nr

In [ ]:
fc2nm_mean_nr = np.array([np.mean(fcntrg2nm_nr['Negative']), np.mean(fcntrg2nm_nr['Positive'])
                        ,np.mean(fcntrg2nm_nr['Polar']),np.mean(fcntrg2nm_nr['Hydrophobic'])
                        , np.mean(fcntrg2nm_nr['Aromatic'])])
fc2nm_mean_nr

In [ ]:
fc2nm_std_nr = np.array([np.std(fcntrg2nm_nr['Negative']), np.std(fcntrg2nm_nr['Positive'])
                       ,np.std(fcntrg2nm_nr['Polar']),np.std(fcntrg2nm_nr['Hydrophobic']), np.std(fcntrg2nm_nr['Aromatic'])])
#fc2nm_std_nr[4] = 0.05
fc2nm_std_nr

In [ ]:
x_pos = np.arange(5)
width = 0.3
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, fc_12nmnr_mean, width, yerr=fc_12nmnr_std, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, fc15nm_mean_nr, width, yerr=fc15nm_std_nr, ecolor='black',capsize=5, color='c')
plt.bar(x_pos+(2*width), fc2nm_mean_nr, width, yerr=fc2nm_std_nr, ecolor='black',capsize=5, color='lightslategray')
plt.title(r'Fractional Contacts Rg unrestrained', fontsize=15)
plt.xticks(x_pos+width, labels=aa_types, fontsize=14)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm', 'Rg = 2 nm'], frameon=False)
plt.ylabel(r'Fractional Contacts', fontsize=15)

### Total fraction of contacts: averages and std dev calc from 5 20ns blocks

In [ ]:
np.mean(fcntrg2nm_nr['total_frac'])

In [ ]:
np.std(fcntrg2nm_nr['total_frac'])

### Avg no. PLGA residues per BSA AA residue group 

In [ ]:
prgrp2nm_nr

In [ ]:
mean_2nm_nr = np.zeros(shape=5)
std_2nm_nr = np.zeros(shape=5)
count = 0
for key, value in prgrp2nm_nr.items():
    mpl_2nm_nr = []
    var_2nm_nr = []
    for i in prgrp2nm_nr[str(key)].flat:
        mpl_2nm_nr.append(i[0])
        var_2nm_nr.append((i[1]))
    
    # calc frac cont averages
    mean_2nm_nr[count] = np.mean(mpl_2nm_nr)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_2nm_nr[count] = np.std(mpl_2nm_nr)
    # std_2nm_nr[count] = np.sqrt(np.sum(var_2nm_nr)/5)
    
    count += 1


In [ ]:
mean_2nm_nr

In [ ]:
#np.std()
#std_2nm_nr[4] = 1
std_2nm_nr

In [ ]:
x_pos = np.arange(5)
width = 0.3
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, mean_12nm_nr, width, yerr=std_12nm_nr, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, mean_15nm_nr, width, yerr=std_15nm_nr, ecolor='black',capsize=5, color='c')
plt.bar(x_pos+(2*width), mean_2nm_nr, width, yerr=std_2nm_nr, ecolor='black',capsize=5, color='lightslategray')
plt.title(r'No. of PLGA residues Rg unrestrained', fontsize=15)
plt.xticks(x_pos+width, labels=aa_types, fontsize=14)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm', 'Rg = 2 nm'], frameon=False)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

### Protein/polymer contact map movie

In [ ]:
trj_ppmap2nm_nr = prot_poly_cntmovie(prot2nm_nores, p2nm_allnr, dmax, pn2nm_nores, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [ ]:
trj_ppmap2nm_nr.shape

In [ ]:
np.save('2nm_NoRes.npy', trj_ppmap2nm_nr)    # .npy extension is added if not given

In [ ]:
trj_load2nm = np.load('2nm_NoRes.npy', allow_pickle=True)
trj_load2nm

In [ ]:
for i in range(1000):
    print(np.nonzero(trj_load2nm[i]))

In [ ]:
trj_load2nm[0].shape

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 2 nm Unres.", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.94, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap2nm_nr[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1.03, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('2nm_NoRes.mp4', writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()